# **FRUIT OBJECT DETECTION USING DEEP LEARNING**

# **IMPORT LIBRARIES**

In [1]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.7 MB/s eta 0:00:00


In [2]:
import cv2
import glob
import shutil
import random
import albumentations as A
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,average_precision_score
from google.colab.patches import cv2_imshow
from PIL import Image
import os
from ultralytics import YOLO
import xml.etree.ElementTree as ET

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# **DATA COLLECTION**

In [6]:
train_images_path="/content/drive/MyDrive/guvi/Data/Train"
test_images_path="/content/drive/MyDrive/guvi/Data/Test"

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os

base = "/content/fruit_dataset"

for p in [
    "images/train", "images/test", "images/valid",
    "labels/train", "labels/test", "labels/valid"
]:
    os.makedirs(os.path.join(base, p), exist_ok=True)

print("Folders created")

Folders created


# **Data Annotation: CONVERT AND COPY IMAGES TO YOLO FORMAT**

In [9]:
import xml.etree.ElementTree as ET

def convert_xml_to_yolo(xml_path, w, h):
    root = ET.parse(xml_path).getroot()
    classes = {"apple":0, "banana":1, "orange":2}
    yolo = []

    for obj in root.findall("object"):
        cls = classes[obj.find("name").text]

        xmin = int(obj.find("bndbox/xmin").text)
        ymin = int(obj.find("bndbox/ymin").text)
        xmax = int(obj.find("bndbox/xmax").text)
        ymax = int(obj.find("bndbox/ymax").text)

        yolo.append(
            f"{cls} "
            f"{(xmin+xmax)/2/w} "
            f"{(ymin+ymax)/2/h} "
            f"{(xmax-xmin)/w} "
            f"{(ymax-ymin)/h}"
        )

    return yolo

# **SPLIT DATASET**

In [11]:
# Load images
train_all = glob.glob("/content/drive/MyDrive/guvi/Data/Train/*.jpg")
test_images = glob.glob("/content/drive/MyDrive/guvi/Data/Test/*jpg")

# Shuffle
random.shuffle(train_all)

# 80-20 split
split = int(0.8 * len(train_all))
train_images = train_all[:split]
valid_images = train_all[split:]

# Print counts
print(len(train_images), "train")
print(len(valid_images), "valid")
print(len(test_images), "test")


192 train
48 valid
60 test


## Data Annotation

In [12]:
def process_split(image_list, mode):
    for image_path in image_list:
        img = cv2.imread(image_path)
        if img is None:
            continue

        h, w = img.shape[:2]

        # Correct XML path handling
        xml_path = os.path.splitext(image_path)[0] + ".xml"
        if not os.path.exists(xml_path):
            continue

        yolo_labels = convert_xml_to_yolo(xml_path, w, h)

        # Create folders safely (train / test / valid)
        os.makedirs(os.path.join(images_folder, mode), exist_ok=True)
        os.makedirs(os.path.join(labels_folder, mode), exist_ok=True)

        # Copy image
        shutil.copy(
            image_path,
            os.path.join(images_folder, mode, os.path.basename(image_path))
        )

        # Write label file
        label_path = os.path.join(
            labels_folder,
            mode,
            os.path.splitext(os.path.basename(image_path))[0] + ".txt"
        )

        with open(label_path, "w") as f:
            f.write("\n".join(yolo_labels))

# **DATA PREPROCESSING:DATA AUGMENTATION**

In [13]:
import os
import cv2

base = "/content/fruit_dataset" # Updated base path

images_folder = os.path.join(base, "images")
labels_folder = os.path.join(base, "labels")

# augmented folders (same level as train)
os.makedirs(os.path.join(images_folder, "train_aug"), exist_ok=True)
os.makedirs(os.path.join(labels_folder, "train_aug"), exist_ok=True)


def save_augmented(image, bboxes, class_labels, img_name):
    # image path
    img_out_path = os.path.join(images_folder, "train_aug", img_name)

    # label path
    label_out_path = os.path.join(
        labels_folder,
        "train_aug",
        os.path.splitext(img_name)[0] + ".txt"
    )

    # save image
    cv2.imwrite(img_out_path, image)

    # save labels
    with open(label_out_path, "w") as f:
        for bbox, cls in zip(bboxes, class_labels):
            f.write(f"{cls} {' '.join(map(str, bbox))}\n")

In [14]:
import os, glob, cv2
import albumentations as A

# Define transform (added missing definition)
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


base = "/content/fruit_dataset" # Updated base path
images_folder = os.path.join(base, "images")
labels_folder = os.path.join(base, "labels")


img_paths = glob.glob(os.path.join(images_folder, "train", "*.jpg"))

for img_path in img_paths:
    name = os.path.basename(img_path)

    txt_path = os.path.join(
        labels_folder, "train",
        name.replace(".jpg", ".txt")
    )

    if not os.path.exists(txt_path):
        continue

    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

    bboxes, labels = [], []
    with open(txt_path) as f:
        for line in f:
            c, x, y, w, h = map(float, line.split())
            labels.append(int(c))
            bboxes.append([x, y, w, h])

    out =  transform(image=img, bboxes=bboxes, class_labels=labels)

    cv2.imwrite(
        os.path.join(images_folder, "train_aug", name),
        cv2.cvtColor(out["image"], cv2.COLOR_RGB2BGR)
    )

    with open(
        os.path.join(labels_folder, "train_aug", name.replace(".jpg", ".txt")),
        "w"
    ) as f:
        for c, (x, y, w, h) in zip(out["class_labels"], out["bboxes"]):
            f.write(f"{c} {x} {y} {w} {h}\n")

In [15]:
import os, glob, cv2

images_folder = "/content/fruit_dataset/images" # Updated path
labels_folder = "/content/fruit_dataset/labels" # Updated path

os.makedirs(images_folder + "/train_aug", exist_ok=True)
os.makedirs(labels_folder + "/train_aug", exist_ok=True)

# collect banana images (class 1)
bana_imgs = []
for img in glob.glob(images_folder + "/train/*.jpg"): # Updated path
    txt = img.replace("images", "labels").replace(".jpg", ".txt")
    if not os.path.exists(txt):
        continue

    with open(txt) as f:
        for line in f:
            if int(line.split()[0]) == 1:
                bana_imgs.append(img)
                break

# augment
for img in bana_imgs:
    txt = img.replace("images", "labels").replace(".jpg", ".txt")
    image = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)


In [16]:
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## **CREATE DATA YAML**

In [17]:
data_yaml = """
path: /content/fruit_dataset
train:
  - images/train
  - images/train_aug
val: images/valid

nc: 3
names:
  - apple
  - banana
  - orange
"""

In [18]:
data_yaml = """
path: /content/fruit_dataset
train:
  - images/train
  - images/train_aug
val: images/valid

nc: 3
names:
  - apple
  - banana
  - orange
"""

In [19]:
with open("/content/data.yaml", "w") as f:
    f.write(data_yaml)

In [20]:
data_yaml = """
path: /content/fruit_dataset
train:
  - images/train
  - images/train_aug
val: images/valid

nc: 3
names:
  - apple
  - banana
  - orange
"""

In [21]:
!ls /content/drive/MyDrive/guvi/Fruit/images/train | head
!ls /content/drive/MyDrive/guvi/Fruit/images/train_aug | head
!ls /content/drive/MyDrive/guvi/Fruit/labels/train | head

ls: cannot access '/content/drive/MyDrive/guvi/Fruit/images/train': No such file or directory
ls: cannot access '/content/drive/MyDrive/guvi/Fruit/images/train_aug': No such file or directory
ls: cannot access '/content/drive/MyDrive/guvi/Fruit/labels/train': No such file or directory


### **MODEL TRAINING:TRAIN YOLOV8**

In [22]:
process_split(train_images, "train")
process_split(valid_images, "valid")
process_split(test_images, "test")


In [23]:
import os

for folder in [images_folder + "/train", images_folder + "/valid", images_folder + "/train_aug"]:
    print(folder, ":", len([f for f in os.listdir(folder) if f.lower().endswith(('.jpg','.png'))]), "images")


/content/fruit_dataset/images/train : 192 images
/content/fruit_dataset/images/valid : 48 images
/content/fruit_dataset/images/train_aug : 0 images


In [24]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

model.train(
    data="/content/data.yaml",
    epochs=50,
    batch=8,
    imgsz=640,
    device=device,
    patience=10
)

Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e9a441e39b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [25]:
import os

base = "/content/fruit_dataset"

for p in [
    "images/train", "images/test", "images/valid",
    "labels/train", "labels/test", "labels/valid"
]:
    os.makedirs(os.path.join(base, p), exist_ok=True)

print("Folders created")

Folders created


# **EVALUATE MODEL**

In [26]:
metrics=model.val(data="/content/data.yaml")
print(metrics)

Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Model summary (fused): 73 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1573.6±966.8 MB/s, size: 119.5 KB)
val: Scanning /content/fruit_dataset/labels/valid.cache... 48 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 48/48 15.5Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.5it/s 2.0s
                   all         48        105      0.881      0.869      0.923      0.665
                 apple         19         34       0.73      0.912      0.876      0.655
                banana         21         41      0.913      0.766      0.907      0.584
                orange         17         30          1      0.928      0.987      0.758
Speed: 6.3ms preprocess, 12.4ms inference, 0.0ms loss, 7.6ms postprocess per image
Results saved to /content/runs/detect/val
ultralytics

In [29]:
!ls -F /content/fruit_dataset/images/valid | head

apple_13.jpg
apple_14.jpg
apple_19.jpg
apple_20.jpg
apple_22.jpg
apple_25.jpg
apple_31.jpg
apple_35.jpg
apple_39.jpg
apple_3.jpg


In [28]:
!ls -F /content/fruit_dataset/images/train | head

apple_10.jpg
apple_11.jpg
apple_12.jpg
apple_15.jpg
apple_16.jpg
apple_17.jpg
apple_18.jpg
apple_1.jpg
apple_21.jpg
apple_23.jpg


In [ ]:
!ls -F /content/fruit_dataset/labels/train_aug | head

In [30]:
summary=metrics.summary()
print(summary)

[{'Class': 'apple', 'Images': np.int64(19), 'Instances': np.int64(34), 'Box-P': np.float64(0.72975), 'Box-R': np.float64(0.91176), 'Box-F1': np.float64(0.81067), 'mAP50': np.float64(0.87601), 'mAP50-95': np.float64(0.65457)}, {'Class': 'banana', 'Images': np.int64(21), 'Instances': np.int64(41), 'Box-P': np.float64(0.91277), 'Box-R': np.float64(0.76586), 'Box-F1': np.float64(0.83289), 'mAP50': np.float64(0.90684), 'mAP50-95': np.float64(0.58356)}, {'Class': 'orange', 'Images': np.int64(17), 'Instances': np.int64(30), 'Box-P': np.float64(1.0), 'Box-R': np.float64(0.92846), 'Box-F1': np.float64(0.9629), 'mAP50': np.float64(0.98651), 'mAP50-95': np.float64(0.7581)}]


In [31]:
metrics.confusion_matrix.matrix


array([[         34,           1,           1,          18],
       [          0,          38,           0,          21],
       [          0,           0,          29,           9],
       [          0,           2,           0,           0]])

# **CUSTOM EVALUATION AND CALCULATE METRICS**

In [32]:
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
import numpy as np

def eval_custom(y_true, y_scores):
    # Compute metrics
    auc_roc = roc_auc_score(y_true, y_scores)
    auc_pr  = average_precision_score(y_true, y_scores)
    y_pred  = (np.array(y_scores) > 0.5).astype(int)
    f1      = f1_score(y_true, y_pred)
    return auc_roc, auc_pr, f1

# Example usage
y_true = [0, 1, 1, 0, 1]
y_scores = [0.1, 0.9, 0.75, 0.2, 0.8]

metrics = eval_custom(y_true, y_scores)
print(metrics)


(np.float64(1.0), np.float64(1.0), 1.0)


# **PREDICTION ON TEST IMAGE**

In [36]:
img=cv2.imread(test_images[2])
results=model(img)
out=results[0].plot()

cv2_imshow(out)




Output hidden; open in https://colab.research.google.com to view.